In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import os
import numpy as np


%matplotlib inline

In [2]:
path = r'E:\Research_Data\Vent_Dyssynchrony\Data\Old_Data\Analyzed_Data\\'


In [3]:
validated_breaths = pd.read_csv(path + 'Validated_Data_v1.csv')


In [4]:
validated_breaths.rename(columns={'Study_ID':'Study ID', 'breath_count':'Breath'}, inplace=True)
validated_breaths.head()

,Study ID,Breath,breath_type,validation
0,P101,40807,DS,d
1,P107,5892,DS,d
2,P101,41164,DS,i
3,P101,21301,DS,"d, f"
4,P101,41641,DS,"d, f"


In [5]:
patients = validated_breaths['Study ID'].unique().tolist()
breath_data = pd.read_csv(path+ 'P100WaveformEdited.csv', header=0, usecols=['Study ID', 'Date_HH:MM:SS', 'Time(ms)', 'Breath'])
t=[]

for patient in patients:
    print(patient)
    temp = pd.read_csv(path + patient + 'WaveformEdited.csv', header=0, usecols=['Study ID', 'Date_HH:MM:SS', 'Time(ms)', 'Breath'])
    t.append(temp)
    
breath_data = pd.concat(t)

P101
P107
P103
P105
P109
P108
P100
P106
P102


In [6]:
breath_data['Study ID'].unique()

array(['P101', 'P107', 'P103', 'P105', 'P109', 'P108', 'P100', 'P106',
       'P102'], dtype=object)

In [7]:
breath_data['Date_Time'] = pd.to_datetime(breath_data['Date_HH:MM:SS'])
breath_data.drop(['Date_HH:MM:SS'], inplace=True, axis=1)

In [8]:
breath_data.shape

(17965580, 4)

In [9]:
validated_breaths.shape

(1224, 4)

In [10]:
grouped = breath_data.groupby(['Study ID', 'Breath'])

In [11]:
grouped.ngroups

206293

In [12]:
simple = grouped.aggregate(np.min)

In [13]:
simple.reset_index(inplace=True)
simple.head()

,Study ID,Breath,Time(ms),Date_Time
0,P100,0,0,2014-04-21 13:01:28
1,P100,1,768,2014-04-21 13:01:29
2,P100,2,4352,2014-04-21 13:01:33
3,P100,3,8320,2014-04-21 13:01:37
4,P100,4,11968,2014-04-21 13:01:40


In [14]:
test = pd.merge(validated_breaths, simple, on=['Study ID', 'Breath'], how='inner')

In [15]:
test.head()

,Study ID,Breath,breath_type,validation,Time(ms),Date_Time
0,P101,40807,DS,d,110465703,2014-04-24 23:35:42
1,P107,5892,DS,d,11779488,2014-10-03 21:26:04
2,P101,41164,DS,i,111455677,2014-04-24 23:52:12
3,P101,21301,DS,"d, f",52264715,2014-04-23 23:31:29
4,P101,41641,DS,"d, f",112872869,2014-04-25 08:15:12


In [16]:
test.drop(['breath_type'], inplace=True, axis=1)

In [17]:
test.head()

,Study ID,Breath,validation,Time(ms),Date_Time
0,P101,40807,d,110465703,2014-04-24 23:35:42
1,P107,5892,d,11779488,2014-10-03 21:26:04
2,P101,41164,i,111455677,2014-04-24 23:52:12
3,P101,21301,"d, f",52264715,2014-04-23 23:31:29
4,P101,41641,"d, f",112872869,2014-04-25 08:15:12


In [18]:
test.validation.describe()

count     1224
unique      14
top          n
freq       388
Name: validation, dtype: object

In [19]:
test.validation.isnull().any()

False

In [20]:
client = MongoClient()
db = client.VentDyssynchrony_db
db_data = db.BreathData_collection
val_data = db.ValidatedData_collection

In [21]:
print(db.collection_names())

['system.indexes', 'VentSettings_collection', 'LogData_collection', 'BreathData_collection', 'LabData_collection', 'RNData_collection', 'RTData_collection', 'PatientData_collection']


In [22]:
db.drop_collection('ValidatedData_collection')

In [23]:
import datetime as dt


def insert_db(df):
    db_data.update({'patientID': df['Study ID'], 'start_time':df.Date_Time.to_pydatetime()},
                   {'$set':{'analysis.validation': df.validation.split(', ')}})

test.apply(insert_db, axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1194    None
1195    None
1196    None
1197    None
1198    None
1199    None
1200    None
1201    None
1202    None
1203    None
1204    None
1205    None
1206    None
1207    None
1208    None
1209    None
1210    None
1211    None
1212    None
1213    None
1214    None
1215    None
1216    None
1217    None
1218    None
1219    None
1220    None
1221    None
1222    None
1223    None
dtype: object

In [24]:
db_data.find({'analysis.validation':{'$exists':1}}).count()

1199

In [25]:
db_data.find_one({'analysis.validation':{'$exists':1}})

{'_id': 'P101\\WF130\\2014-04-22 18:43:02',
 'analysis': {'validation': ['d', 'f']},
 'breath_number': 130,
 'characteristics': {'breath_time': 0.9920000000000186,
  'end_insp_vol': 402.5,
  'exp_time': 0.22399999999998954,
  'insp_time': 0.7680000000000291,
  'max_flow': "[[37.200000000000003, '0 days 00:04:56.148000', 8], [46.5, '0 days 00:04:56.692000', 25]]",
  'max_pressure': "[[19.399999999999999, '0 days 00:04:56.628000', 23]]",
  'max_vol': 445.0,
  'min_exp_flow': 46.2,
  'min_flow': "[[4.5, '0 days 00:04:56.756000', 27]]",
  'min_pressure': "[[18.600000000000001, '0 days 00:04:56.468000', 18]]",
  'min_vol': -2.5,
  'peak_insp_flow': 46.5,
  'peak_pressure': 21.7},
 'data_frame': {'DateTime': [datetime.datetime(2014, 4, 22, 18, 43, 2),
   datetime.datetime(2014, 4, 22, 18, 43, 2),
   datetime.datetime(2014, 4, 22, 18, 43, 2),
   datetime.datetime(2014, 4, 22, 18, 43, 2),
   datetime.datetime(2014, 4, 22, 18, 43, 2),
   datetime.datetime(2014, 4, 22, 18, 43, 2),
   datetime.da

In [27]:
db_data.aggregate([{'$match':{'analysis.validation':{'$exists':1}}},
                   {'$out': 'ValidatedData_collection'}])

In [30]:
val_data.find({'analysis.validation':{'$exists':1}}).count()

1199

In [36]:
for items in val_data.find({'analysis.validation':{'$exists':1}},{'_id':0, 'patientID':1, 'file_name':1, 'breath_number':1}):
    try:
        val_data.insert_one(db_data.find_one({'patientID':items['patientID'], 'file_name':items['file_name'], 
                                              'breath_number':items['breath_number']+1}))
    except as e:
        print(e)

TypeError: document must be an instance of dict, bson.son.SON, or other type that inherits from collections.MutableMapping

In [37]:
val_data.find({'analysis.validation':{'$exists':1}}).count()

1199

In [38]:
val_data.find().count()

1560